# Assignmnet 2 (100 points)

**Name:** Abdullah Zeeshan<br>
**Email:** abz3929@thi.de<br>
**Group:** B <br>
**Hours spend *(optional)* :** <br>

### SMS Spam Detection *(60 points)*

<p>You are hired as an AI expert in the development department of a telecommunications company. The first thing on your orientation plan is a small project that your boss has assigned you for the following given situation. Your supervisor has given away his private cell phone number on too many websites and is now complaining about daily spam SMS. Therefore, it is your job to write a spam detector in Python. </p>

<p>In doing so, you need to use a Naive Bayes classifier that can handle both bag-of-words (BoW) and tf-idf features as input. For the evaluation of your spam detector, an SMS collection is available as a dataset - this has yet to be suitably split into train and test data. To keep the costs as low as possible and to avoid problems with copyrights, your boss insists on a new development with Python.</p>

<p>Include a short description of the data preprocessing steps, method, experiment design, hyper-parameters, and evaluation metric. Also, document your findings, drawbacks, and potential improvements.</p>

<p>Note: You need to implement the bag-of-words (BoW) and tf-idf feature extractor from scratch. You can use existing python libraries for other tasks.</p>

**Dataset and Resources**

* SMS Spam Collection Dataset: https://archive.ics.uci.edu/dataset/228/sms+spam+collection

In [ ]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Download necessary NLTK data (run once)
try:
    nltk.data.find('corpora/stopwords')
except nltk.downloader.DownloadError:
    nltk.download('stopwords')
try:
    nltk.data.find('tokenizers/punkt')
except nltk.downloader.DownloadError:
    nltk.download('punkt')
try:
    nltk.data.find('tokenizers/punkt_tab')
except nltk.downloader.DownloadError:
    nltk.download('punkt_tab')

def load_data(file_path):
    messages = []
    labels = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            parts = line.split(None, 1)
            label = parts[0].strip()
            message = parts[1].strip()
            messages.append(message)
            labels.append(label)

    return messages, labels

def preprocess(messages):
    messages_lower = [message.lower() for message in messages]
    stop_words = set(stopwords.words('english'))
    porter = PorterStemmer()
    processed_messages = []
    for message in messages_lower:
        tokens = nltk.word_tokenize(message)
        alphanumeric_tokens = [token for token in tokens if re.match(r'[a-zA-Z0-9]+$', token)]
        tokens_no_stopwords = [token for token in alphanumeric_tokens if token not in stop_words]
        stemmed_tokens = [porter.stem(token) for token in tokens_no_stopwords]
        processed_messages.append(stemmed_tokens)
    return processed_messages

def bag_of_words(documents):
    num_documents = len(documents)
    vocabulary_size = len(vocabulary)
    bow_matrix = [[0 for _ in range(vocabulary_size)] for _ in range(num_documents)]
    word_to_index = {word: i for i, word in enumerate(vocabulary)}
    for i, document in enumerate(documents):
        for word in document:
            if word in word_to_index:
                index = word_to_index[word]
                bow_matrix[i][index] += 1

    return bow_matrix

def tf_idf(documents):
    num_documents = len(documents)  
    vocabulary_size = len(vocabulary)  
    tfidf_matrix = [[0.0 for _ in range(vocabulary_size)] for _ in range(num_documents)]
    word_to_index = {word: i for i, word in enumerate(vocabulary)}

    # Calculate term frequency (TF)
    tf_matrix = [[0.0 for _ in range(vocabulary_size)] for _ in range(num_documents)]
    for i, document in enumerate(documents):
        for word in document:
            if word in word_to_index:
                index = word_to_index[word]
                tf_matrix[i][index] += 1

    # Calculate document frequency (DF)
    document_frequency = [0.0] * vocabulary_size
    for i, document in enumerate(documents):
        for document in documents:
            if word in document:
                document_frequency[i] += 1

    # Calculate inverse document frequency (IDF)
    inverse_document_frequency = [0.0] * vocabulary_size
    for i in range(vocabulary_size):
        if document_frequency[i] > 0:
            inverse_document_frequency[i] = np.log(num_documents / document_frequency[i])

    # Calculate TF-IDF
    for i in range(num_documents):
        for j in range(vocabulary_size):
            if tf_matrix[i][j] > 0:
                tfidf_matrix[i][j] = tf_matrix[i][j] * inverse_document_frequency[j]

    return tfidf_matrix


# Load the dataset
file_path = 'sms_spam_collection\SMSSpamCollection'
messages, labels = load_data(file_path)

# Preprocess the messages  
processed_messages = preprocess(messages)

# Create the vocabulary
all_words = [word for tokens in processed_messages for word in tokens]
vocabulary = list(set(all_words))

# Create the Bag-of-Words features
bow_features = bag_of_words(processed_messages)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    np.array(bow_features),  # Convert to NumPy array
    labels,
    test_size=0.2,
    random_state=42,
    stratify=labels
)

# Train a Naive Bayes classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred_bow = nb_classifier.predict(X_test)

# Evaluate the model
print("\nEvaluation of Naive Bayes classifier on BoW features:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_bow):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_bow, pos_label='spam'):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_bow, pos_label='spam'):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_bow, pos_label='spam'):.4f}")


# Create the TF-IDF features
tfidf_features = tf_idf(processed_messages)

# Split data into training and testing sets for TF-IDF
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(
    np.array(tfidf_features),  # Convert to NumPy array
    labels,
    test_size=0.2,
    random_state=42,
    stratify=labels
)

# Train a Naive Bayes classifier on TF-IDF features
nb_classifier_tfidf = MultinomialNB()
nb_classifier_tfidf.fit(X_train_tfidf, y_train_tfidf)

# Make predictions on the test set for TF-IDF
y_pred_tfidf = nb_classifier_tfidf.predict(X_test_tfidf)

# Evaluate the model on TF-IDF features
print("\nEvaluation of Naive Bayes classifier on TF-IDF features:")
print(f"Accuracy: {accuracy_score(y_test_tfidf, y_pred_tfidf):.4f}")
print(f"Precision: {precision_score(y_test_tfidf, y_pred_tfidf, pos_label='spam'):.4f}")
print(f"Recall: {recall_score(y_test_tfidf, y_pred_tfidf, pos_label='spam'):.4f}")
print(f"F1 Score: {f1_score(y_test_tfidf, y_pred_tfidf, pos_label='spam'):.4f}")

## You can use sklearn or other python libraries for naive bayes classifier, evaluation metric, etc.  ##


Evaluation of Naive Bayes classifier on BoW features:
Accuracy: 0.9758
Precision: 0.9067
Recall: 0.9128
F1 Score: 0.9097

Evaluation of Naive Bayes classifier on TF-IDF features:
Accuracy: 0.9435
Precision: 0.7263
Recall: 0.9262
F1 Score: 0.8142


## Spam Detection Experiment Details

Here's a breakdown of the experiment, including data preprocessing, methodology, design, hyperparameters, evaluation, findings, drawbacks, and potential improvements:

**1. Data Preprocessing**

The dataset consists of SMS messages labeled as either "ham" (legitimate) or "spam". The following preprocessing steps are applied:

* Lowercasing: Converts text to lowercase.
* Tokenization: Splits text into words (tokens).
* Alphanumeric Filtering: Removes non-alphanumeric tokens.
* Stop Word Removal: Eliminates common words (e.g., "the", "and", "is").
* Stemming: Reduces words to their root form (e.g., "running" to "run").

**2. Method**

The code uses the Multinomial Naive Bayes algorithm for text classification. The approach involves:

* Feature Extraction: Converting preprocessed messages into numerical vectors using:
    * Bag-of-Words (BoW): Word frequency vectors.
    * TF-IDF: Weights words by frequency and inverse document frequency.
* Model Training: Training a Multinomial Naive Bayes classifier.
* Prediction: Predicting labels (ham or spam) for test data.
* Evaluation: Assessing performance with metrics.

**3. Experiment Design**

The experiment:

1.  Loads the SMS Spam Collection dataset.
2.  Applies preprocessing steps.
3.  Creates a vocabulary of unique words for BoW and TF-IDF.
4.  Extracts BoW and TF-IDF features.
5.  Splits data into 80/20 training/testing sets with stratification.
6.  Trains and evaluates Naive Bayes on both feature sets.
7.  Compares performance.

**4. Hyperparameters**

The Multinomial Naive Bayes classifier uses:

* `alpha` (Smoothing parameter): Laplace smoothing for zero-frequency words (default: 1.0).

**5. Evaluation Metrics**

Performance is evaluated using:

* Accuracy: Proportion of correctly classified messages.
* Precision: Proportion of correctly classified spam out of all predicted spam.
* Recall: Proportion of correctly classified spam out of all actual spam.
* F1-score: Balanced measure of precision and recall.

**6. Findings**

The code evaluates the Multinomial Naive Bayes classifier on both BoW and TF-IDF representations of the text data. Here are the results:

* **Evaluation of Naive Bayes classifier on BoW features:**
    * Accuracy: 0.9758
    * Precision: 0.9067
    * Recall: 0.9128
    * F1 Score: 0.9097
* **Evaluation of Naive Bayes classifier on TF-IDF features:**
    * Accuracy: 0.9435
    * Precision: 0.7263
    * Recall: 0.9262
    * F1 Score: 0.8142

As the results indicate, the Naive Bayes classifier performs better with BoW features than with TF-IDF features on this dataset. BoW achieves higher accuracy, precision, recall, and F1-score.

**7. Drawbacks**

* Simplicity of Naive Bayes: Assumes conditionally independent features, often violated in text.
* Vocabulary Dependence: Performance depends on training data vocabulary; struggles with out-of-vocabulary words.
* Limited Context: Ignores word order.

**8. Potential Improvements**

* Advanced Preprocessing: Explore lemmatization or n-grams.
* Feature Engineering: Use word embeddings (e.g., Word2Vec) or metadata features (message length, sender info).
* Model Selection: Experiment with SVMs, Logistic Regression, ensemble methods, or deep learning.
* Hyperparameter Tuning: Optimize the Naive Bayes smoothing parameter and other model parameters.
* Larger Dataset: Train on more diverse data.
* Evaluation on Diverse Datasets: Assess performance across different spam datasets.


 ### Search Engine *(40 points)*

Your boss is impressed with your spam detector and assigns you a new task. As part of improving internal tools, the company wants a search engine that can search through SMS messages and rank them by relevance. Implement the PageRank algorithm from scratch to score each SMS message based on its importance in the document graph.

*   Compute TF-IDF vectors for all SMS messages (you can leverage previous implementation)
*   Construct a document graph, where each node represents an SMS message and edges are the links between nodes.
*  Implement the PageRank algorithm from scratch to assign an importance score to each SMS message based on its position in the document graph.

#### Hint : You can use the previous dataset or any dataset from your choice.




## You might need the follwoing formulas for your implementation

---

### 1) Cosine Similarity Between Two Document Vectors

Cosine similarity measures how similar two vectors are based on the angle between them:

$$
\text{cosine\_sim}(A, B) = \frac{A \cdot B}{\|A\| \cdot \|B\|}
$$

- \( A \cdot B \): Dot product of vectors \( A \) and \( B \)  
- \( \|A\| \): Euclidean norm (magnitude) of vector \( A \)  
- \( \|B\| \): Euclidean norm of vector \( B \)

**Use case**: Comparing TF-IDF vectors to measure similarity between two messages.

---

### 2) PageRank of a Node \( i \)

PageRank estimates the importance of a document based on its connections in a graph:

$$
PR(i) = \frac{1 - d}{N} + d \sum_{j \in M(i)} \frac{PR(j)}{L(j)}
$$

Where:
- \( PR(i) \): PageRank score of node \( i \)  
- \( d \): Damping factor (typically 0.85)  
- \( N \): Total number of nodes (documents) in the graph  
- \( M(i) \): Set of nodes that link to node \( i \)  
- \( L(j) \): Number of outbound links from node \( j \)  

**Interpretation**:  
- A document is important if **important documents link to it**.  
- The score is split among a node’s outbound links.  
- The **teleportation term** $\text(\frac{1 - d}{N})$ accounts for random jumps, ensuring stability and fairness.

---



In [16]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import nltk

def load_data(filepath):
    messages = []
    with open(filepath, 'r', encoding='utf-8') as file:
        for line in file:
            parts = line.split(None, 1)
            if len(parts) == 2:
                messages.append(parts[1].strip())
    return messages

def preprocess(messages):
    stop_words = set(stopwords.words('english'))
    porter = PorterStemmer()
    processed_messages = []
    for message in messages:
        tokens = nltk.word_tokenize(message.lower())
        alphanumeric_tokens = [token for token in tokens if re.match(r'[a-zA-Z0-9]+$', token)]
        tokens_no_stopwords = [token for token in alphanumeric_tokens if token not in stop_words]
        stemmed_tokens = [porter.stem(token) for token in tokens_no_stopwords]
        processed_messages.append(stemmed_tokens)
    return processed_messages

def tf_idf(documents):
    num_documents = len(documents)
    vocabulary = sorted(list(set(word for doc in documents for word in doc)))
    vocabulary_size = len(vocabulary)
    word_to_index = {word: i for i, word in enumerate(vocabulary)}

    tf_matrix = np.zeros((num_documents, vocabulary_size))
    for i, document in enumerate(documents):
        for word in document:
            if word in word_to_index:
                index = word_to_index[word]
                tf_matrix[i, index] += 1

    document_frequency = np.zeros(vocabulary_size)
    for i, word in enumerate(vocabulary):
        for document in documents:
            if word in document:
                document_frequency[i] += 1

    idf = np.log(num_documents / (document_frequency + 1))
    tfidf_matrix = tf_matrix * idf
    return tfidf_matrix


def construct_graph(tfidf_matrix):
    similarity_matrix = cosine_similarity(tfidf_matrix)
    return similarity_matrix

def pagerank(graph, damping_factor=0.85, max_iterations=100):
    num_nodes = len(graph)  # Number of nodes in the graph
    node_ranks = [1.0 / num_nodes] * num_nodes  # Initialize ranks to 1/N for each node

    for iteration in range(max_iterations):
        new_node_ranks = [0.0] * num_nodes  # Store updated ranks for each node

        for i in range(num_nodes):  # Iterate through each node i
            for j in range(num_nodes):  # Iterate through each node j to find incoming links to i
                if graph[j][i] > 0:  # If there's a link from node j to node i
                    new_node_ranks[i] += node_ranks[j] / sum(graph[j])  # Add j's contribution to i's rank

        for i in range(num_nodes):
            new_node_ranks[i] = (1 - damping_factor) / num_nodes + damping_factor * new_node_ranks[i]\
            
        node_ranks = new_node_ranks  # Update ranks for the next iteration

    return node_ranks  # Return the final PageRank scores

def normalize_pagerank_scores(pagerank_scores):
    """Normalizes PageRank scores so they sum to 1."""
    total_score = sum(pagerank_scores)
    normalized_scores = [score / total_score for score in pagerank_scores]
    return normalized_scores

filepath = 'sms_spam_collection\Reduced_SMSSpamCollection'
messages = load_data(filepath)
print("Loaded data.")  # Print statement
processed_messages = preprocess(messages)
print("Preprocessed messages.")  # Print statement
tfidf_matrix = tf_idf(processed_messages)
print("Calculated TF-IDF matrix.")  # Print statement
graph = construct_graph(tfidf_matrix)
print("Constructed graph.")  # Print statement
ranks = pagerank(graph)
normalized_scores = normalize_pagerank_scores(ranks)
print("Calculated PageRank scores.")  # Print statement

# Print results (most relevant messages first)
message_ranks = list(zip(messages, normalized_scores))
message_ranks.sort(key=lambda x: x[1], reverse=True)
print("PageRank Results:")
for i, (message, rank) in enumerate(message_ranks):
    print(f"Rank {i + 1}: Score = {rank:.4f}, Message = {message[:100]}...")

Loaded data.
Preprocessed messages.
Calculated TF-IDF matrix.
Constructed graph.
Calculated PageRank scores.
PageRank Results:
Rank 1: Score = 0.0170, Message = Hmmm.. Thk sure got time to hop ard... Ya, can go 4 free abt... Muz call u to discuss liao......
Rank 2: Score = 0.0162, Message = You'll not rcv any more msgs from the chat svc. For FREE Hardcore services text GO to: 69988 If u ge...
Rank 3: Score = 0.0148, Message = HEY GIRL. HOW R U? HOPE U R WELL ME AN DEL R BAK! AGAIN LONG TIME NO C! GIVE ME A CALL SUM TIME FROM...
Rank 4: Score = 0.0148, Message = Hi. Wk been ok - on hols now! Yes on for a bit of a run. Forgot that i have hairdressers appointment...
Rank 5: Score = 0.0144, Message = I'm ok wif it cos i like 2 try new things. But i scared u dun like mah. Cos u said not too loud....
Rank 6: Score = 0.0139, Message = Hi its Kate how is your evening? I hope i can see you tomorrow for a bit but i have to bloody babyjo...
Rank 7: Score = 0.0139, Message = For real when u gettin

## PageRank Implementation Summary

Here's a summary of what the code does, the thinking behind it, the findings, and what the PageRank tells us:

### Workflow

1.  **Load Data:**
    * SMS messages are loaded from the `Reduced_SMSSpamCollection` file. The `load_data` function reads this file and extracts the message content, ignoring the labels.

2.  **Preprocess Messages:**
    * Messages are converted to lowercase.
    * Stop words are removed.
    * Words are stemmed to their root form.  The `preprocess` function cleans and standardizes the text data.

3.  **Calculate TF-IDF Matrix:**
    * The TF-IDF (Term Frequency-Inverse Document Frequency) matrix is calculated.
    * TF-IDF measures word importance.
    * This step transforms the text into a numerical representation.

4.  **Construct Graph:**
    * A graph is constructed where:
        * Each SMS message is a node.
        * Edges represent cosine similarity between TF-IDF vectors.
    * The graph represents message relationships based on word usage.

5.  **Calculate PageRank Scores:**
    * The PageRank algorithm is applied to the graph.
    * This assigns an "importance" score to each message.
    * PageRank ranks messages based on their connections in the graph.

6.  **Normalize PageRank Scores:**
    * Raw PageRank scores are normalized.
    * This creates a probability distribution.

7.  **Print Results:**
    * Messages are ranked by normalized PageRank scores.
    * The output displays the most "important" messages.

### Thinking Behind It

* The code uses PageRank to identify "important" SMS messages, similar to how PageRank was used to rank web pages.
* A message's importance is determined by its similarity to other important messages.
* TF-IDF represents message content, enabling similarity calculation.
* The graph representation allows PageRank to find influential messages.

### Findings

* The code calculates a PageRank score for each message, indicating its relative importance.
* Messages similar to many other messages receive higher PageRank scores.
* The output ranks messages by their PageRank scores.

### What Does PageRank Tell Us?

* PageRank identifies "central" or "influential" messages.
* A high PageRank score suggests similarity to many other messages.
* In an SMS dataset:
    * Messages in a common theme or conversation may rank higher.
    * Spam campaigns with similar messages may also rank higher.
    * Unique or isolated messages may rank lower.


### Additional Experiments *(5 additional points - <span style="color: red;">Optional</span>)*